In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import time
from utils import somCluster

startTime = time.time()

# Loading Dataset and preprocessing

x = pd.read_csv('DataSet/indianPines_X.csv')
y = pd.read_csv('DataSet/indianPines_Y.csv')
columns = list(x.columns)
X = []
Y = []
nFeatures = x.shape[1]
sc = MinMaxScaler(feature_range=(0, 1))
x = sc.fit_transform(x)
nClasses = len(y.value_counts())

# Splitting the data set and performing

df = []
for i in range(nClasses):
    df.append(x[y['class'] == i])

for i in range(nClasses):
    if i == 0:
        X = somCluster(df[i], nFeatures)
        Y = [i] * len(X)
    else:
        tp = somCluster(df[i], nFeatures)
        X = np.concatenate((X, tp), axis=0)
        Y = np.concatenate((Y, [i] * len(tp)), axis=0)

# Dataset without Outliers
X = pd.DataFrame(X, columns=columns)
Y = pd.DataFrame(Y, columns=['class'])
print(X.shape[0])

# K-MEANS HOMOGENEOUS ##########################################################
df = pd.concat([X, Y], axis=1, join='inner')
df = df.sample(frac=1).reset_index(drop=True)
print(df.head(20))
print('Runtime:', time.time() - startTime)

11052
           X         Y         1         2         3         4         5  \
0   0.791667  0.347222  0.000000  0.543328  0.305310  0.429899  0.282222   
1   0.763889  0.791667  0.307186  0.379572  0.202434  0.304617  0.110776   
2   0.986111  0.673611  0.214069  0.420099  0.165929  0.320664  0.073046   
3   0.041667  0.375000  0.091599  0.514003  0.349558  0.471002  0.299427   
4   0.027778  0.479167  0.006073  0.469193  0.175885  0.344313  0.149110   
5   0.409722  0.965278  0.319838  0.463262  0.244838  0.406532  0.205554   
6   0.826389  0.243056  0.383097  0.415815  0.168510  0.343468  0.093269   
7   0.840278  0.784722  0.385628  0.333443  0.198746  0.300957  0.077573   
8   0.020833  0.750000  0.570850  0.595058  0.349189  0.475225  0.301238   
9   0.500000  0.465278  0.321356  0.505437  0.343289  0.471002  0.267129   
10  0.076389  0.895833  0.126012  0.304778  0.177729  0.284065  0.111379   
11  0.826389  0.069444  0.385628  0.419769  0.202802  0.329673  0.169031   
12  0.

In [10]:
from sklearn.cluster import KMeans
queue = [df]
final = []
# p = list(df['class'].value_counts().keys())
# print(p)
while len(queue) > 0:
    cluster = queue.pop(0)
    if len(cluster['class'].value_counts()) == 1:
        final.append(list(cluster.mean()))
    else:
        classCentroid = []
        classes = list(cluster['class'].value_counts().keys())
        for i in classes:
            centroid = list(cluster[cluster['class'] == i].mean())
            classCentroid.append(centroid)
        kMeans = KMeans(n_clusters=len(classes), init=np.array(classCentroid), n_init=1, max_iter=500)
        label = kMeans.fit_predict(cluster)
        for i in range(len(classes)):
            queue.append(cluster[label==i])

## Final Reduced Dataset
print(len(final))

47
